In [1]:
import sys
sys.path.insert(0, '../../Models/ANN')

In [1]:
# Import libraries
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from ANN import ANN
import pandas as pd
import hyperopt
from hyperopt import fmin, tpe, Trials, hp
import hyperopt.pyll.stochastic
from hyperopt import pyll

# Hyperopt function
def search_reg(space, max_eval, output_path):
    T = Trials()
    best = fmin(optimiser, space, algo=tpe.suggest, max_evals=max_eval, trials=T, pass_expr_memo_ctrl=True)
    
    logs = []
    results = []
    
    T.trials
    for i in T.trials:
        logs.append(i['misc']['vals'])
        results.append(i['result']['loss'])
    
    trial_data = pd.DataFrame(logs)
    trial_data['F1'] = results
    trial_data.to_csv(output_path)
    
    return hyperopt.space_eval(space, best)

In [2]:
x = pd.read_csv('../../Data/Missing_0_Mask/X_train_0.csv')
y = pd.read_csv('../../Data/Y_train.csv')
xt = pd.read_csv('../../Data/Missing_0_Mask/X_test_0.csv')
yt = pd.read_csv('../../Data/Y_test.csv')

In [3]:
def Kfold(data, labels, shape=[26,12], activation_fn=None, dropout=0, optim = 'adamw', lr = 0.001, weight_decay = 0.1, n_epoch = 100, batchsize = 1024, momentum = None, run_name = 'testrun', best = None):
    
    kf = StratifiedKFold(n_splits = 5, shuffle = True)
    kf.get_n_splits(data, labels)
    
    VS_arr = []
    
    for fold, (train_index, test_index) in enumerate(kf.split(data, labels)):
        print('--------------------------------')
        print(f'FOLD {fold}')
        print('--------------------------------')
        
        x_train, x_test = data.iloc[train_index], data.iloc[test_index]
        y_train, y_test = labels.iloc[train_index], labels.iloc[test_index]

        Model = ANN(shape, activation_fn, dropout)
        Model.fit( x=x_train, y=y_train, xt = x_test, yt = y_test, optim = optim, lr = lr, weight_decay = weight_decay, n_epoch = n_epoch, batchsize = batchsize, momentum = momentum, run_name = run_name)
        f1 = f1_score(yt,Model.predict(xt).cpu().numpy(),average='macro')
        VS_arr.append(f1)
        
        print(f'Fold {fold} Validation Score: {f1}')
        
        # Early Stopping
        if best != None:
            if f1 < best:
                print('********************************')
                print('F1 Score too low ... stopped early.')
                print('********************************')
                return f1

    
    final_F1 = min(VS_arr)
    print('--------------------------------')
    print(f'\n\nMinimum Validation Score: {final_F1}')
    print('--------------------------------')
    
    return final_F1

In [5]:
space = {'activation_fn':hp.choice('activation_fn', ['relu', 'leaky_relu', 'sigmoid', 'tanh']),
         'neurons':hp.randint('neurons',500),
         'dropout':hp.uniform('dropout', 0, 1)}

In [6]:
def optimiser(expr, memo, ctrl):
    try:
        config = pyll.rec_eval(expr, memo=memo)
        print(config)
        if len(ctrl.trials.losses()) > 1:
            best_score = -min(value for value in ctrl.trials.losses() if value is not None)
        else:
            best_score = -5

        return -Kfold(x, y, shape=[26, config['neurons'],12], activation_fn=config['activation_fn'], dropout=config['dropout'], optim = 'adamw', lr = 0.0001, weight_decay = 0.1, n_epoch = 10, batchsize = 256, momentum = None, run_name = '1_layer_search', best = best_score)
    
    except:
        return 99

In [7]:
search_reg(space, 100, '1_layer.csv')

{'activation_fn': 'tanh', 'dropout': 0.9889743181110722, 'neurons': 187}
--------------------------------                     
FOLD 0                                               
--------------------------------                     
Device is cuda:0.                                    

  0%|          | 0/100 [00:04<?, ?it/s, best loss: ?]

wandb: Currently logged in as: 2djagpal (use `wandb login --relogin` to force relogin)



Loading Data ...                                     

Starting training...                                 

Training:                                            
Epoch: 1 ... Accuracy: 0.6136822837908262 ... Score: -1.1579224814397142

Validation:                                          
Epoch: 1 ... Accuracy: 0.6136529647206571 ... Score: -1.1580810583375694

Training:                                            
Epoch: 2 ... Accuracy: 0.6140571204005092 ... Score: -1.1573943996633946

Validation:                                          
Epoch: 2 ... Accuracy: 0.6140374108832437 ... Score: -1.1574616728534022

Training:                                            
Epoch: 3 ... Accuracy: 0.6143337092378536 ... Score: -1.156605881197085

Validation:                                          
Epoch: 3 ... Accuracy: 0.6143620543094279 ... Score: -1.156547545311252

Training:                                            
Epoch: 4 ... Accuracy: 0.6142610913191686 ... Score: -1.15647386075300

train_accuracy,0.61514
train_batch_loss,1.87637
train_epoch_loss,0.0
train_score,-1.15456
valid_accuracy,0.61513
valid_epoch_loss,2e-05
valid_score,-1.15458


Loading Data ...                                     

Starting training...                                 

Training:                                            
Epoch: 1 ... Accuracy: 0.6124481930438516 ... Score: -1.1629438899028095

Validation:                                          
Epoch: 1 ... Accuracy: 0.6125022426121947 ... Score: -1.1625391923178785

Training:                                            
Epoch: 2 ... Accuracy: 0.6142049040536774 ... Score: -1.1575398677287383

Validation:                                          
Epoch: 2 ... Accuracy: 0.6142749741565642 ... Score: -1.1571670682010406

Training:                                            
Epoch: 3 ... Accuracy: 0.6148595325333268 ... Score: -1.1552917848931397

Validation:                                          
Epoch: 3 ... Accuracy: 0.6149199921401782 ... Score: -1.1550600806486062

Training:                                            
Epoch: 4 ... Accuracy: 0.6149641876573164 ... Score: -1.154977953009

...

train_accuracy,0.6279
train_batch_loss,0.67102
train_epoch_loss,0.0
train_score,-1.0055
valid_accuracy,0.62748
valid_epoch_loss,1e-05
valid_score,-1.00575


Loading Data ...                                                                   

Starting training...                                                               

Training:                                                                          
Epoch: 1 ... Accuracy: 0.4894148704410898 ... Score: -1.5183916893063707           

Validation:                                                                        
Epoch: 1 ... Accuracy: 0.4903439938830344 ... Score: -1.5155257514854572           

Training:                                                                          
Epoch: 2 ... Accuracy: 0.545500465609008 ... Score: -1.3141742167943888            

Validation:                                                                        
Epoch: 2 ... Accuracy: 0.546460318748585 ... Score: -1.3115338120399995            

Training:                                                                          
Epoch: 3 ... Accuracy: 0.5786238477244962 ... Score: -1.20420839527214

train_accuracy,0.68113
train_batch_loss,1.10127
train_epoch_loss,0.0
train_score,-0.88278
valid_accuracy,0.6817
valid_epoch_loss,2e-05
valid_score,-0.88276


Loading Data ...                                                                   

Starting training...                                                               

Training:                                                                          
Epoch: 1 ... Accuracy: 0.42056667606427967 ... Score: -1.7207730230839549          

Validation:                                                                        
Epoch: 1 ... Accuracy: 0.42136580906694915 ... Score: -1.7195379811450515          

Training:                                                                          
Epoch: 2 ... Accuracy: 0.44031341039376 ... Score: -1.636961933259861              

Validation:                                                                        
Epoch: 2 ... Accuracy: 0.4403446346266387 ... Score: -1.6381656792095787           

Training:                                                                          
Epoch: 3 ... Accuracy: 0.45385238058624017 ... Score: -1.5778042263628

{'activation_fn': 'tanh', 'dropout': 0.053341907762284424, 'neurons': 199}

In [8]:
space = {'activation_fn':hp.choice('activation_fn', ['relu', 'leaky_relu', 'sigmoid', 'tanh']),
         'neurons':hp.randint('neurons',500),
         'neurons2':hp.randint('neurons2',500),
         'dropout':hp.uniform('dropout', 0, 1)}

In [9]:
def optimiser(expr, memo, ctrl):
    try:
        config = pyll.rec_eval(expr, memo=memo)
        print(config)
        if len(ctrl.trials.losses()) > 1:
            best_score = -min(value for value in ctrl.trials.losses() if value is not None)
        else:
            best_score = -5

        return -Kfold(x, y, shape=[26, config['neurons'], config['neurons2'],12], activation_fn=config['activation_fn'], dropout=config['dropout'], optim = 'adamw', lr = 0.0001, weight_decay = 0.1, n_epoch = 10, batchsize = 256, momentum = None, run_name = '2_layer_search', best = best_score)
    
    except:
        return 99

In [10]:
search_reg(space, 100, '2_layer.csv')

{'activation_fn': 'tanh', 'dropout': 0.11155604033229638, 'neurons': 135, 'neurons2': 21}
--------------------------------                     
FOLD 0                                               
--------------------------------                     
Device is cuda:0.                                    

  0%|          | 0/100 [00:00<?, ?it/s, best loss: ?]

train_accuracy,0.57239
train_batch_loss,0.82499
train_epoch_loss,0.0
train_score,-1.19019
valid_accuracy,0.57299
valid_epoch_loss,1e-05
valid_score,-1.18817


Loading Data ...                                     

Starting training...                                 

Training:                                            
Epoch: 1 ... Accuracy: 0.6332496091447318 ... Score: -1.056948199929945

Validation:                                          
Epoch: 1 ... Accuracy: 0.6333067068768875 ... Score: -1.0563186930539121

Training:                                            
Epoch: 2 ... Accuracy: 0.6600509606923478 ... Score: -0.9407304294709143

Validation:                                          
Epoch: 2 ... Accuracy: 0.6592781809716236 ... Score: -0.9419838703476675

Training:                                            
Epoch: 3 ... Accuracy: 0.6764796968842641 ... Score: -0.8905122019461602

Validation:                                          
Epoch: 3 ... Accuracy: 0.6758520822031329 ... Score: -0.8907857865982067

Training:                                            
Epoch: 4 ... Accuracy: 0.6845007731672519 ... Score: -0.8636120686709

train_accuracy,0.69715
train_batch_loss,0.91008
train_epoch_loss,0.0
train_score,-0.80848
valid_accuracy,0.69739
valid_epoch_loss,1e-05
valid_score,-0.80813


Loading Data ...                                     

Starting training...                                 

Training:                                            
Epoch: 1 ... Accuracy: 0.6211377720632758 ... Score: -1.1150904679472324

Validation:                                          
Epoch: 1 ... Accuracy: 0.620152753927775 ... Score: -1.1185167149362243

Training:                                            
Epoch: 2 ... Accuracy: 0.6532167033849525 ... Score: -0.9887479723069728

Validation:                                          
Epoch: 2 ... Accuracy: 0.6523865665393718 ... Score: -0.9916222415870005

Training:                                            
Epoch: 3 ... Accuracy: 0.6744093659928514 ... Score: -0.9090019692249861

Validation:                                          
Epoch: 3 ... Accuracy: 0.6741633988603258 ... Score: -0.9102469222817404

Training:                                            
Epoch: 4 ... Accuracy: 0.6852849556123446 ... Score: -0.8659821723200

...

train_accuracy,0.6865
train_batch_loss,0.92515
train_epoch_loss,0.0
train_score,-0.84457
valid_accuracy,0.68697
valid_epoch_loss,1e-05
valid_score,-0.84358


Loading Data ...                                                                  

Starting training...                                                              

Training:                                                                         
Epoch: 1 ... Accuracy: 0.4326297938505438 ... Score: -1.4955723092925306          

Validation:                                                                       
Epoch: 1 ... Accuracy: 0.4346249300521565 ... Score: -1.4903899138413434          

Training:                                                                         
Epoch: 2 ... Accuracy: 0.6076293666863162 ... Score: -1.1542000121741804          

Validation:                                                                       
Epoch: 2 ... Accuracy: 0.6073993071425825 ... Score: -1.1537106530031653          

Training:                                                                         
Epoch: 3 ... Accuracy: 0.616793107278024 ... Score: -1.1339492240561806          

train_accuracy,0.64526
train_batch_loss,1.61153
train_epoch_loss,0.0
train_score,-0.97129
valid_accuracy,0.64408
valid_epoch_loss,1e-05
valid_score,-0.97363


Loading Data ...                                                                  

Starting training...                                                              

Training:                                                                         
Epoch: 1 ... Accuracy: 0.611895669409061 ... Score: -1.1685752898309285           

Validation:                                                                       
Epoch: 1 ... Accuracy: 0.6117734501480118 ... Score: -1.1692230556635328          

Training:                                                                         
Epoch: 2 ... Accuracy: 0.4340447753543327 ... Score: -1.7742015232676356          

Validation:                                                                       
Epoch: 2 ... Accuracy: 0.4327326006074249 ... Score: -1.7782455799370362          

Training:                                                                         
Epoch: 3 ... Accuracy: 0.49114168183099677 ... Score: -1.5786151709511238        

{'activation_fn': 'leaky_relu',
 'dropout': 0.08052031029452844,
 'neurons': 229,
 'neurons2': 491}

In [11]:
space = {'activation_fn':hp.choice('activation_fn', ['relu', 'leaky_relu', 'sigmoid', 'tanh']),
         'neurons':hp.randint('neurons',500),
         'neurons2':hp.randint('neurons2',500),
         'neurons3':hp.randint('neurons3',500),
         'dropout':hp.uniform('dropout', 0, 1)}

In [12]:
def optimiser(expr, memo, ctrl):
    try:
        config = pyll.rec_eval(expr, memo=memo)
        print(config)
        if len(ctrl.trials.losses()) > 1:
            best_score = -min(value for value in ctrl.trials.losses() if value is not None)
        else:
            best_score = -5

        return -Kfold(x, y, shape=[26, config['neurons'], config['neurons2'], config['neurons3'],12], activation_fn=config['activation_fn'], dropout=config['dropout'], optim = 'adamw', lr = 0.0001, weight_decay = 0.1, n_epoch = 10, batchsize = 256, momentum = None, run_name = '3_layer_search', best = best_score)
    
    except:
        return 99

In [13]:
search_reg(space, 100, '3_layer.csv')

{'activation_fn': 'leaky_relu', 'dropout': 0.12352887325478379, 'neurons': 17, 'neurons2': 113, 'neurons3': 15}
--------------------------------                     
FOLD 0                                               
--------------------------------                     
Device is cuda:0.                                    

  0%|          | 0/100 [00:00<?, ?it/s, best loss: ?]

train_accuracy,0.61581
train_batch_loss,1.69108
train_epoch_loss,1e-05
train_score,-1.15295
valid_accuracy,0.61582
valid_epoch_loss,2e-05
valid_score,-1.15291


Loading Data ...                                     

Starting training...                                 

Training:                                            
Epoch: 1 ... Accuracy: 0.41480636645564756 ... Score: -1.7980834742975285

Validation:                                          
Epoch: 1 ... Accuracy: 0.41380076291205153 ... Score: -1.800716992093224

Training:                                            
Epoch: 2 ... Accuracy: 0.6112154103766734 ... Score: -1.1601625573467975

Validation:                                          
Epoch: 2 ... Accuracy: 0.6116837460434082 ... Score: -1.158390751079653

Training:                                            
Epoch: 3 ... Accuracy: 0.6153556996522883 ... Score: -1.1549172476100162

Validation:                                          
Epoch: 3 ... Accuracy: 0.615327441339923 ... Score: -1.1551288321807067

Training:                                            
Epoch: 4 ... Accuracy: 0.6159078094164082 ... Score: -1.1516497082468

train_accuracy,0.69406
train_batch_loss,1.55047
train_epoch_loss,0.0
train_score,-0.85863
valid_accuracy,0.69391
valid_epoch_loss,1e-05
valid_score,-0.85937


Loading Data ...                                     

Starting training...                                 

Training:                                            
Epoch: 1 ... Accuracy: 0.4341607139615275 ... Score: -1.565958224451068

Validation:                                          
Epoch: 1 ... Accuracy: 0.432555040110721 ... Score: -1.5707672296691186

Training:                                            
Epoch: 2 ... Accuracy: 0.49993218775129245 ... Score: -1.396909897277792

Validation:                                          
Epoch: 2 ... Accuracy: 0.5001708656910235 ... Score: -1.397443208515946

Training:                                            
Epoch: 3 ... Accuracy: 0.5240861632043263 ... Score: -1.337201879734176

Validation:                                          
Epoch: 3 ... Accuracy: 0.525001922239024 ... Score: -1.335977479901923

Training:                                            
Epoch: 4 ... Accuracy: 0.5789425347257449 ... Score: -1.1820660096175923



...

train_accuracy,0.63874
train_batch_loss,0.76096
train_epoch_loss,0.0
train_score,-0.98636
valid_accuracy,0.63894
valid_epoch_loss,1e-05
valid_score,-0.98625


Loading Data ...                                                                   

Starting training...                                                               

Training:                                                                          
Epoch: 1 ... Accuracy: 0.5493005727198265 ... Score: -1.2941429706463736           

Validation:                                                                        
Epoch: 1 ... Accuracy: 0.5488974891286704 ... Score: -1.2947427830603755           

Training:                                                                          
Epoch: 2 ... Accuracy: 0.5741433497542207 ... Score: -1.2234696590912733           

Validation:                                                                        
Epoch: 2 ... Accuracy: 0.5737029158230174 ... Score: -1.2254999957283577           

Training:                                                                          
Epoch: 3 ... Accuracy: 0.5366469139019382 ... Score: -1.35143470961940

train_accuracy,0.63117
train_batch_loss,0.81779
train_epoch_loss,0.0
train_score,-1.01314
valid_accuracy,0.63045
valid_epoch_loss,1e-05
valid_score,-1.01573


Loading Data ...                                                                   

Starting training...                                                               

Training:                                                                          
Epoch: 1 ... Accuracy: 0.5961535997129457 ... Score: -1.2133263491982127           

Validation:                                                                        
Epoch: 1 ... Accuracy: 0.5961948373151988 ... Score: -1.2132875272849986           

Training:                                                                          
Epoch: 2 ... Accuracy: 0.6172854140502858 ... Score: -1.1387068991294393           

Validation:                                                                        
Epoch: 2 ... Accuracy: 0.6165832988043725 ... Score: -1.1412118810950735           

Training:                                                                          
Epoch: 3 ... Accuracy: 0.6335261979820762 ... Score: -1.07526606991824

{'activation_fn': 'tanh',
 'dropout': 0.043873613092414464,
 'neurons': 457,
 'neurons2': 383,
 'neurons3': 71}